In [1]:
import backtrader as bt
import talib as ta
import pandas as pd
import json
from futu import *
import time
import numpy as np
import math
import datetime
import pandas as pd
import backtrader as bt
import backtrader.indicators as btind
import matplotlib
matplotlib.use('nbagg')  # 适用于 Jupyter 的交互式后端
import matplotlib.pyplot as plt  # 导入 matplotlib 确保显示
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 300

## 策略一
BBI 上升（均线多头排列允许一定回撤） + 短/长期 RSV 条件（最近m天长期RSV全>=80,短期RSV曾到过20以下且短期RSV头尾都大于80） + MACD DIF > 0 
## 策略二
放量突破 + KDJ + DIF>0 + 收盘价波动幅度
条件：1、价格波动约束（排除价格小于0且波动幅度超过阈值的标的）
2、J值条件：当日J值小于阈值且小于等于其历史分位数阈值
3、当日DIF必须大于0
4、寻找突破日：单日涨幅必须大于阈值，其他所有日成交量必须小于等于突破日成交量的倍数，且当日价格创新高。突破日后的所有日J值大于最后一日J值-10。




In [9]:
target_pools = ['SH.688521', 'SH.600030', 'SH.600760', 'SH.600895', 'SH.600580',\
    'SZ.002236', 'SH.688041', 'SH.601127', 'SH.688195', 'SZ.300499', 'SZ.301128', 'SH.603201']
columns = ['code', 'name', 'update_time', 'last_price', 'open_price', 'high_price', \
    'low_price', 'pe_ratio', 'volume', 'turnover', 'turnover_rate']

In [2]:
df = pd.read_parquet('data/kline_data.parquet')

In [4]:
df.sort_values(by='time_key', ascending=False, inplace=True)

In [44]:
df.head()

,0,code,name,time_key,open,close,high,low,pe_ratio,volume,turnover_rate,turnover,change_rate
0,None,SZ.300499,高澜股份,2020-01-02,7.727978,7.767978,7.807978,7.694644,38.051,2374329.0,0.01285,18833224.00,0.2581
1,None,SZ.300499,高澜股份,2020-01-03,7.727978,7.674644,7.807978,7.627978,37.603,2142885.0,0.01159,16861133.17,-1.2015
2,None,SZ.300499,高澜股份,2020-01-06,7.674644,7.754644,7.867978,7.614644,37.987,2983290.0,0.01614,23643045.54,1.0424
3,None,SZ.300499,高澜股份,2020-01-07,7.754644,8.207978,8.214644,7.754644,40.159,6169414.0,0.03338,50450599.05,5.8460
4,None,SZ.300499,高澜股份,2020-01-08,8.121311,8.201311,8.407978,7.994644,40.127,6434400.0,0.03481,53673482.62,-0.0812


In [324]:
class MultiIndicatorStrategy(bt.Strategy):
    params = (
        ('short_window', 3),    # 短期均线
        ('mid_window', 6),     # 中期均线
        ('long_window', 12),    # 长期均线
        ('longer_window', 24),  # 更长期均线
        ('period', 9),  # %K计算周期
        ('period_dfast', 3),  # %D（快速）计算周期
        ('period_dslow', 3),  # %D（慢速）计算周期，对应KDJ中的%D
        ('m_days', 90),          # 最近m天
        ('j_threshold', 0.1),
        ('j_q_threshold', 0.05),
        ('fastperiod', 12),   # 快速EMA周期
        ('slowperiod', 26),   # 慢速EMA周期
        ('bbi_min_window', 30),  # BBI最小窗口
        ('bbi_q_threshold', 0.1), # 允许的最大回撤
        ('printlog', False),    # 是否打印日志
    )

    def __init__(self):
        # 均线指标
        self.short_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.short_window)
        self.mid_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.mid_window)
        self.long_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.long_window)
        self.longer_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.longer_window)
        self.bbi = (self.short_ma+self.mid_ma+self.long_ma+self.longer_ma)/4
        self.bbi_cnt = 0
        self.kdj_cnt = 0
        self.macd_cnt = 0

        # KDJ指标(RSV计算)
        self.stoch = bt.indicators.Stochastic(
            self.data,
            period=self.params.period,  # 基础周期
            period_dfast=self.params.period_dfast,  # %K的平滑周期
            period_dslow=self.params.period_dslow,  # %D的平滑周期
            movav=bt.ind.MovAv.SMA  # 平滑方式，默认SMA
        )
        self.k = self.stoch.percK
        # KDJ中的%D对应Stochastic的percD
        self.d = self.stoch.percD
        # 计算%J线（公式：J = 3*K - 2*D）
        self.j = 3 * self.k - 2 * self.d
        # self.j_history = []
        # MACD指标
        self.dif = bt.indicators.EMA(self.data.close, period=self.params.fastperiod) - bt.indicators.EMA(self.data.close, period=self.params.slowperiod)
        self.crossover = bt.indicators.CrossOver(
            self.data.close, 
            self.mid_ma
        )
        
        # 交易记录
        self.order = None
        self.buyprice = 0
        self.buycomm = 0
        self.buyvolumn = 0


    def log(self, txt, dt=None, doprint=False):
        """日志记录"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        """订单状态处理"""
        if order.status in [order.Submitted, order.Accepted]:
            return

        # 订单完成
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'买入, 价格: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手续费: {order.executed.comm:.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # 卖出
                self.log(f'卖出, 价格: {order.executed.price:.2f}, 收入: {order.executed.value:.2f}, 手续费: {order.executed.comm:.2f}')
            
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单取消/保证金不足/拒绝')

        self.order = None

    def notify_trade(self, trade):
        """交易状态处理"""
        if not trade.isclosed:
            return

        self.log(f'交易利润, 总利润: {trade.pnl:.2f}, 净利润: {trade.pnlcomm:.2f}')
    def bbi_deriv_uptrend(self, 
        bbi: pd.Series,
        *,
        min_window: int,
        max_window: int | None = None,
        q_threshold: float = 0.0,
    ) -> bool:
        if not 0.0 <= q_threshold <= 1.0:
            raise ValueError("q_threshold 必须位于 [0, 1] 区间内")
        bbi_data = []
        for i in range(-self.params.m_days, 1):  # 索引范围：-n, -(n-1), ..., 0
            if isinstance(bbi, float):
                continue
            else:
                bbi_val = bbi[i]
            if not math.isnan(bbi_val):
                bbi_data.append(bbi_val)
        # print(f"bbi_data: {len(bbi_data)}")
        bbi = pd.Series(bbi_data).dropna()

        if len(bbi) < min_window:
            return False

        longest = min(len(bbi), max_window or len(bbi))

        # 自最长窗口向下搜索，找到任一满足条件的区间即通过
        for w in range(longest, min_window - 1, -1):
            seg = bbi.iloc[-w:]           # 区间 [T-w+1, T]
            norm = seg / seg.iloc[0]           # 归一化
            diffs = np.diff(norm.values)       # 一阶差分
            if np.quantile(diffs, q_threshold) >= 0:
                return True
        return False
    def next(self):
        """策略逻辑实现"""
        if self.order:
            return

        # 检查是否已经持仓
        position = self.getposition(self.data)
        current_size = position.size
        # 均线多头排列: 短期 > 中期 > 长期 > 更长期
        ma_condition = self.bbi_deriv_uptrend(
            self.bbi, 
            min_window=self.params.bbi_min_window, 
            max_window=self.params.m_days, 
            q_threshold=self.params.bbi_q_threshold)
        
        current_j = self.j[0]
        if math.isnan(current_j): 
            return
        j_windows = []
        for i in range(-self.params.m_days, 1):
            j_windows.append(self.j[i])

        # 历史数据量不足时，不执行判断
        if len(j_windows) < self.params.m_days:
            self.log(f"历史J值数据不足（当前{len(j_windows)}/{self.params.m_days}）")
            return
        
        # 计算J值的历史分位数（例如5%分位）
        j_quantile = np.percentile(j_windows, self.params.j_q_threshold * 100)
        # 条件判断：J < 阈值 或 J <= 历史分位值
        condition1 = current_j < self.params.j_threshold
        condition2 = current_j <= j_quantile
        kdj_trigger = condition1 or condition2

        # MACD DIF>0
        macd_condition = self.dif[0] > 0
        self.macd_cnt += macd_condition
        self.kdj_cnt += kdj_trigger
        self.bbi_cnt += ma_condition
        # self.log(f"macd_cnt: {self.macd_cnt}, rsv_cnt: {self.kdj_cnt}, bbi_cnt: {self.bbi_cnt}")

        # 所有条件都满足时买入
        if ma_condition and kdj_trigger and macd_condition:
            self.log(f'买入信号, 价格: {self.data.close[0]:.2f}')
            self.order = self.buy(size=400)

            # 简单的卖出条件: 短期均线下穿中期均线
        # if (self.data.close[0] < self.short_ma[0]) and (current_size>0):
        if (self.crossover<0) and (self.position):
            self.log(f'价格跌破5日线, 卖出信号, 价格: {self.data.close[0]:.2f}')
            self.order = self.sell(size=current_size)

    def stop(self):
        """回测结束时输出最终资产"""
        self.log(f'最终资产价值: {self.broker.getvalue():.2f}', doprint=True)

In [287]:
def run_backtest(stock_code, start_date, end_date, initial_cash=100000):
    # 创建回测引擎
    cerebro = bt.Cerebro()
    
    # 添加策略
    cerebro.addstrategy(MultiIndicatorStrategy, printlog=True)
    
    # 获取数据
    df = pd.read_parquet('data/kline_data.parquet')
    df = df[df['code'] == stock_code]
    df['time_key'] = pd.to_datetime(df['time_key'])
    df.set_index('time_key', inplace=True)
    df = df[start_date:end_date]
    df = df[['open', 'high', 'low', 'close', 'volume']]


    
    # 转换为backtrader数据格式
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    
    # 设置初始资金
    cerebro.broker.setcash(initial_cash)
    
    # 设置手续费
    cerebro.broker.setcommission(commission=0.001)  # 0.1%手续费
    
    # 添加分析指标
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
    
    print(f'初始资金: {initial_cash:.2f}')
    
    # 运行回测
    results = cerebro.run()
    strat = results[0]
    
    # 输出分析结果
    print(f'最终资金: {cerebro.broker.getvalue():.2f}')
    print(f'总收益率: {strat.analyzers.returns.get_analysis()["rtot"]:.2%}')
    print(f'夏普比率: {strat.analyzers.sharpe.get_analysis()["sharperatio"]:.2f}')
    print(f'最大回撤: {strat.analyzers.drawdown.get_analysis()["max"]["drawdown"]:.2%}')
    
    # 交易统计
    trade_analysis = strat.analyzers.trade_analyzer.get_analysis()
    if 'total' in trade_analysis:
        print(f'总交易次数: {trade_analysis["total"]["total"]}')
        print(f'盈利交易次数: {trade_analysis["won"]["total"]}')
        print(f'亏损交易次数: {trade_analysis["lost"]["total"]}')
        if trade_analysis["won"]["total"] > 0:
            print(f'平均盈利: {trade_analysis["won"]["pnl"]["average"]:.2f}')
        if trade_analysis["lost"]["total"] > 0:
            print(f'平均亏损: {trade_analysis["lost"]["pnl"]["average"]:.2f}')
    
    # 绘制回测结果
    fig = cerebro.plot()[0][0]
    fig.set_size_inches(12, 8)
    fig.savefig('test.png')

    return results

In [288]:
run_backtest('SZ.002236', '2022-01-01', '2025-08-21')

初始资金: 100000.00
2023-04-17, 买入信号, 价格: 22.43
2023-04-18, 买入, 价格: 22.43, 成本: 8973.15, 手续费: 8.97
2023-04-18, 买入信号, 价格: 23.22
2023-04-19, 买入, 价格: 23.03, 成本: 9213.15, 手续费: 9.21
2023-04-21, 价格跌破5日线, 卖出信号, 价格: 23.38
2023-04-24, 卖出, 价格: 23.45, 收入: 18186.31, 手续费: 18.76
2023-04-24, 交易利润, 总利润: 576.00, 净利润: 539.05
2025-08-21, 最终资产价值: 100539.05
最终资金: 100539.05
总收益率: 0.54%
夏普比率: -3.71
最大回撤: 142.03%
总交易次数: 1
盈利交易次数: 1
亏损交易次数: 0
平均盈利: 539.05


<IPython.core.display.Javascript object>

In [226]:
df = pd.read_parquet('data/kline_data.parquet')
df[df['code']=='SH.688041']

,code,name,time_key,open,close,high,low,pe_ratio,volume,turnover_rate,turnover,change_rate
6683,SH.688041,海光信息,2022-08-12,69.68037,59.78037,73.48037,59.78037,165.564,119133625,0.59684,7.864878e+09,67.5441
6684,SH.688041,海光信息,2022-08-15,56.68037,56.50037,60.87037,56.38037,156.528,59475630,0.29797,3.509521e+09,-5.4868
6685,SH.688041,海光信息,2022-08-16,55.68037,60.37037,61.34037,54.69037,167.190,45226222,0.22658,2.650931e+09,6.8495
6686,SH.688041,海光信息,2022-08-17,60.03037,57.32037,61.54037,56.48037,158.787,30965039,0.15513,1.814533e+09,-5.0521
6687,SH.688041,海光信息,2022-08-18,56.70037,57.33037,58.47037,55.18037,158.815,20379982,0.10210,1.172482e+09,0.0174
...,...,...,...,...,...,...,...,...,...,...,...,...
7404,SH.688041,海光信息,2025-08-19,152.10000,148.73000,153.66000,148.70000,151.610,36311641,0.01562,5.472755e+09,-2.5424
7405,SH.688041,海光信息,2025-08-20,148.88000,155.79000,155.96000,146.77000,158.807,48236173,0.02075,7.339021e+09,4.7469
7406,SH.688041,海光信息,2025-08-21,156.91000,155.05000,160.30000,154.14000,158.053,41757541,0.01797,6.561111e+09,-0.4750
7407,SH.688041,海光信息,2025-08-22,159.18000,186.06000,186.06000,159.18000,189.663,75154632,0.03233,1.330998e+10,20.0000


In [ ]:
class BreakoutVolumeKDJStrategy(bt.Strategy):
    params = (
        ('j_threshold', 0.0),
        ('up_threshold', 3.0),
        ('volume_threshold', 2.0/3),
        ('offset', 15),
        ('max_window', 60),
        ('price_range_pct', 80.0),
        ('j_q_threshold', 0.20),
        ('printlog', False),    # 是否打印日志
    )

    def __init__(self):
        # 1. 初始化技术指标
        self.kdj = bt.indicators.StochasticSlow(
            period=9, period_dfast=3, period_dslow=3
        )
        self.k = self.kdj.percK
        # KDJ中的%D对应Stochastic的percD
        self.d = self.kdj.percD
        # 计算%J线（公式：J = 3*K - 2*D）
        self.j = 3 * self.k - 2 * self.d
        self.dif = bt.indicators.MACD().macd - bt.indicators.MACD().signal
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=5)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=20)
        # 2. 存储中间计算结果
        self.j_values = []
        self.price_high = bt.indicators.Highest(self.data.high, period=self.p.max_window)
        self.price_low = bt.indicators.Lowest(self.data.low, period=self.p.max_window)
        self.crossover = bt.indicators.CrossOver(
            self.data.close, 
            self.short_ma
        )
        self.pct_change = bt.indicators.PercentChange(self.data.close, period=1)

    def next(self):
        # 1. 基础数据收集
        current_date = self.data.datetime.date(0)
        close = self.data.close[0]
        volume = self.data.volume[0]
        j_value = self.j[0]  # 使用慢速K线近似J值
        dif_value = self.dif[0]
        # 2. 收盘价波动幅度约束
        price_range = (self.price_high[0] / self.price_low[0] - 1) * 100
        if price_range > self.p.price_range_pct:
            return

        # 3. J值分位计算
        self.j_values.append(j_value)
        if len(self.j_values) < self.p.max_window:
            # print("窗口数据不足")
            return

        j_series = pd.Series(self.j_values[-self.p.max_window:])
        j_quantile = j_series.quantile(self.p.j_q_threshold)

        # 4. J值条件和DIF条件
        j_condition = (j_value < self.p.j_threshold) or (j_value <= j_quantile)
        if not j_condition or dif_value <= 0:
            return

        # 实现offset周期内只要满足以下四个条件就生成买入信号：
        # 1) 单日涨幅 ≥ up_threshold（默认3%）
        # 2) 相对放量：突破日成交量 ≥ 其他日的1/volume_threshold倍
        # 3) 创新高：突破日收盘价 > 之前所有收盘价
        # 4) J值维持高位：突破后J值未大幅回落（> 当前J值-10）
        buy_signal = False
        breakout_date = None
        # print("计算循环条件")
        for i in range(-self.p.offset-1, 0):
            condition1 = self.pct_change[i]*100 >= self.p.up_threshold
            if not condition1:
                continue
            # print("条件一满足")
            # 2) 相对放量
            vol_T = self.data.volume[i]
            if vol_T <= 0:
                continue
            
            vols_except_T = [self.data.volume[x] for x in range(-self.p.max_window, 0) if x != i]
            for item in vols_except_T:
                if item > self.p.volume_threshold * vol_T:
                    continue
            # print("条件二满足")
            # 3) 创新高
            tmp = [self.data.close[x] for x in range(-self.p.max_window, 0)]
            if self.data.close[i] > max(tmp):
                continue
            # print("条件三满足")
            # 4) J值维持高位
            for idx in range(i, 0):
                if self.j[idx] <= j_value-10:
                    continue
            # print("条件四满足")
            buy_signal = True
            breakout_date = current_date
            break
        

        # 6. 生成买入信号
        if buy_signal == True and (self.broker.get_cash() >= 200 * self.data.close[0]):
            self.buy(size=200)
            self.log(f'买入: 价格={close:.2f}, 成交量={volume}')
                
        if (self.crossover<0) and (self.position):
            current_size = self.position.size
            self.log(f'价格跌破5日线, 卖出信号, 价格: {self.data.close[0]:.2f}')
            self.order = self.sell(size=current_size)

    def log(self, txt, dt=None):
        dt = dt or self.data.datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')


In [16]:
def run_backtest(strategy, stock_code, start_date, end_date, initial_cash=100000):
    # 创建回测引擎
    cerebro = bt.Cerebro()
    
    # 添加策略
    cerebro.addstrategy(strategy, printlog=True)
    
    # 获取数据
    df = pd.read_parquet('data/kline_data.parquet')
    df = df[df['code'] == stock_code]
    if df.shape[0] == 0:
        print("数据为空")
        return None
    print(f"stock name: {df['name'].iloc[0]}")
    df['time_key'] = pd.to_datetime(df['time_key'])
    df.set_index('time_key', inplace=True)
    df = df[start_date:end_date]
    df = df[['open', 'high', 'low', 'close', 'volume']]


    
    # 转换为backtrader数据格式
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    
    # 设置初始资金
    cerebro.broker.setcash(initial_cash)
    
    # 设置手续费
    cerebro.broker.setcommission(commission=0.001)  # 0.1%手续费
    
    # 添加分析指标
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
    
    print(f'初始资金: {initial_cash:.2f}')
    
    # 运行回测
    results = cerebro.run()
    strat = results[0]
    
    # 输出分析结果
    print(f'最终资金: {cerebro.broker.getvalue():.2f}')
    print(f'总收益率: {strat.analyzers.returns.get_analysis()["rtot"]:.2%}')
    # print(f'夏普比率: {strat.analyzers.sharpe.get_analysis()["sharperatio"]:.2f}')
    print(f'最大回撤: {strat.analyzers.drawdown.get_analysis()["max"]["drawdown"]:.2%}')
    
    # 交易统计
    trade_analysis = strat.analyzers.trade_analyzer.get_analysis()
    if 'total' in trade_analysis:
        print(f'总交易次数: {trade_analysis["total"]["total"]}')
        if 'won' in trade_analysis['total']:
            print(f'盈利交易次数: {trade_analysis["won"]["total"]}')
        if 'lost' in trade_analysis['total']:
            print(f'亏损交易次数: {trade_analysis["lost"]["total"]}')
        # if trade_analysis["won"]["total"] > 0:
        #     print(f'平均盈利: {trade_analysis["won"]["pnl"]["average"]:.2f}')
        # if trade_analysis["lost"]["total"] > 0:
        #     print(f'平均亏损: {trade_analysis["lost"]["pnl"]["average"]:.2f}')
    
    # 绘制回测结果
    fig = cerebro.plot()[0][0]
    fig.set_size_inches(12, 8)
    fig.savefig('test.png')

    return results

In [49]:
class BBIKDJStrategy(bt.Strategy):
    params = (
        ('j_threshold', -5),          # J值绝对阈值
        ('bbi_min_window', 30),       # BBI最小窗口
        ('max_window', 120),           # 最大窗口周期
        ('price_range_pct', 100.0),   # 价格波动阈值(百分比)
        ('bbi_q_threshold', 0.1),    # BBI分位阈值
        ('j_q_threshold', 0.10),      # J值分位阈值
        ('printlog', False),          # 是否打印日志
    )

    def __init__(self):
        # 1. 初始化技术指标
        # BBI指标 (3,6,12,24日均线)
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=3)
        self.mid_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=6)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=12)
        self.longer_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=24)
        self.bbi = (self.short_ma + self.mid_ma + self.long_ma + self.longer_ma) / 4

        # KDJ指标
        self.stoch = bt.indicators.Stochastic(
            period=9, period_dfast=3, period_dslow=3, movav=bt.ind.MovAv.SMA
        )
        self.k = self.stoch.percK
        self.d = self.stoch.percD
        self.j = 3 * self.k - 2 * self.d  # J值计算

        # MACD指标
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=12,
            period_me2=26,
            period_signal=9
        )
        self.dif = self.macd.macd - self.macd.signal

        # 价格波动指标
        self.price_high = bt.indicators.Highest(self.data.high, period=self.p.max_window)
        self.price_low = bt.indicators.Lowest(self.data.low, period=self.p.max_window)

        # 交易状态管理
        self.order = None
        self.buyprice = 0
        self.buycomm = 0

    def log(self, txt, dt=None, doprint=False):
        """日志记录"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        """订单状态处理"""
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'买入, 价格: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手续费: {order.executed.comm:.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'卖出, 价格: {order.executed.price:.2f}, 收入: {order.executed.value:.2f}, 手续费: {order.executed.comm:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单取消/保证金不足/拒绝')
        self.order = None

    def notify_trade(self, trade):
        """交易状态处理"""
        if not trade.isclosed:
            return
        self.log(f'交易利润, 总利润: {trade.pnl:.2f}, 净利润: {trade.pnlcomm:.2f}')

    def bbi_deriv_uptrend(self, bbi_series, min_window=30):
        """BBI趋势判断"""
        if len(bbi_series) < min_window:
            return False
        # 计算归一化后的BBI斜率分位数
        norm = bbi_series / bbi_series.iloc[0]
        diffs = np.diff(norm.values)
        return np.quantile(diffs, self.p.bbi_q_threshold) >= 0

    def next(self):
        """策略主逻辑"""
        if self.order:
            return

        # 1. 基础数据收集
        close = self.data.close[0]
        volume = self.data.volume[0]

        # 2. 收盘价波动幅度约束
        win_high = self.price_high[0]
        win_low = self.price_low[0]
        if win_low <= 0:
            return
        price_range = (win_high / win_low - 1)*100
        if price_range > self.p.price_range_pct:
            return

        # 3. BBI趋势判断
        bbi_values = [self.bbi[i] for i in range(-self.p.max_window, 0) if not math.isnan(self.bbi[i])]
        if not self.bbi_deriv_uptrend(pd.Series(bbi_values), min_window=self.p.bbi_min_window):
            return
        else:
            self.log(f'BBI趋势判断, 价格: {close:.2f}')
        # 4. KDJ J值条件
        j_values = [self.j[i] for i in range(-self.p.max_window, 0) if not math.isnan(self.j[i])]
        if len(j_values) < self.p.max_window:
            return
        j_quantile = np.percentile(j_values, self.p.j_q_threshold * 100)
        j_condition = self.j[0] < self.p.j_threshold or self.j[0] <= j_quantile
        if not j_condition:
            return
        else:
            self.log(f"J yes: {self.j[0]}")
        # 5. MACD DIF > 0条件
        if self.dif[0] <= 0:
            return

        # 6. 生成买入信号
        if not self.position:
            cash_needed = 200 * close
            if self.broker.get_cash() >= cash_needed:
                self.log(f'买入信号触发, 价格: {close:.2f}')
                self.order = self.buy(size=200)


In [ ]:
for item in target_pools:
    # run_backtest(BreakoutVolumeKDJStrategy, item, '2022-01-01', '2025-08-21')
    run_backtest(BBIKDJStrategy, item, '2022-01-01', '2025-08-21')

In [2]:
data = { "SH.512150": { "signal": "buy_to_enter", "quantity": 15000, "leverage": 1, "profit_target": 30000.0, "stop_loss": 28200.0, "confidence": 0.7, "justification": "价格高于SMA且RSI适中，显示健康上升趋势。" }, "SH.516390": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.5, "justification": "趋势中性，RSI适中，缺乏强劲信号。" }, "SH.515210": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.4, "justification": "价格下跌且RSI偏低，趋势疲弱，避免进入。" }, "SH.515790": { "signal": "buy_to_enter", "quantity": 30000, "leverage": 1, "profit_target": 30300.0, "stop_loss": 28500.0, "confidence": 0.65, "justification": "价格上涨并高于SMA，RSI中性，有上升潜力。" }, "SH.512760": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.6, "justification": "趋势强但RSI较高，接近超买，需谨慎观察。" }, "SH.510300": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.6, "justification": "趋势好但RSI较高，等待回调或更好入场点。" }, "SZ.159605": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.5, "justification": "价格和RSI中性，无明显趋势信号。" }, "SH.516300": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.55, "justification": "趋势向上但RSI适中，可作为备选，当前持仓已满。" }, "SH.561910": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.55, "justification": "趋势好但RSI适中，缺乏独特优势，暂不进入。" }, "SH.515980": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.3, "justification": "RSI超买，风险高，避免追高。" }, "SZ.159887": { "signal": "buy_to_enter", "quantity": 30000, "leverage": 1, "profit_target": 42000.0, "stop_loss": 39900.0, "confidence": 0.5, "justification": "RSI超卖，可能反弹机会，但趋势弱，高风险高回报。" }, "SH.512880": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.6, "justification": "趋势向上但RSI较高，需监控超买信号。" }, "SH.588000": { "signal": "buy_to_enter", "quantity": 20000, "leverage": 1, "profit_target": 29600.0, "stop_loss": 27800.0, "confidence": 0.7, "justification": "价格高于SMA，RSI适中偏高，显示持续上升趋势。" }, "SZ.159880": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.3, "justification": "RSI严重超买，价格高位，风险极大，避免进入。" }, "SH.513130": { "signal": "hold", "quantity": 0, "leverage": 1, "profit_target": 0.0, "stop_loss": 0.0, "confidence": 0.5, "justification": "趋势和RSI中性，无明确交易信号。" } }

In [6]:
pd.DataFrame(data).T.reset_index().rename(columns={'index': 'code'})

,code,signal,quantity,leverage,profit_target,stop_loss,confidence,justification
0,SH.512150,buy_to_enter,15000,1,30000.0,28200.0,0.7,价格高于SMA且RSI适中，显示健康上升趋势。
1,SH.516390,hold,0,1,0.0,0.0,0.5,趋势中性，RSI适中，缺乏强劲信号。
2,SH.515210,hold,0,1,0.0,0.0,0.4,价格下跌且RSI偏低，趋势疲弱，避免进入。
3,SH.515790,buy_to_enter,30000,1,30300.0,28500.0,0.65,价格上涨并高于SMA，RSI中性，有上升潜力。
4,SH.512760,hold,0,1,0.0,0.0,0.6,趋势强但RSI较高，接近超买，需谨慎观察。
5,SH.510300,hold,0,1,0.0,0.0,0.6,趋势好但RSI较高，等待回调或更好入场点。
6,SZ.159605,hold,0,1,0.0,0.0,0.5,价格和RSI中性，无明显趋势信号。
7,SH.516300,hold,0,1,0.0,0.0,0.55,趋势向上但RSI适中，可作为备选，当前持仓已满。
8,SH.561910,hold,0,1,0.0,0.0,0.55,趋势好但RSI适中，缺乏独特优势，暂不进入。
9,SH.515980,hold,0,1,0.0,0.0,0.3,RSI超买，风险高，避免追高。
